In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import glob
import random
import xgboost as xgb
import lightgbm as lgb

from sklearn.metrics import matthews_corrcoef, r2_score, mean_squared_error, f1_score, fbeta_score
from sklearn import preprocessing
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier, CatBoostRegressor

from rgf.sklearn import RGFClassifier, RGFRegressor
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

from sklearn.svm import SVC, SVR
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from math import sqrt
from scipy import stats
from scipy.stats import norm, skew
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, Ridge
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
os.chdir(r'/Users/gympohnpimol/Desktop/Hackathons')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')
train.head(10)

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0
5,58896,Analytics,region_2,Bachelor's,m,sourcing,2,31,3.0,7,0,0,85,0
6,20379,Operations,region_20,Bachelor's,f,other,1,31,3.0,5,0,0,59,0
7,16290,Operations,region_34,Master's & above,m,sourcing,1,33,3.0,6,0,0,63,0
8,73202,Analytics,region_20,Bachelor's,m,other,1,28,4.0,5,0,0,83,0
9,28911,Sales & Marketing,region_1,Master's & above,m,sourcing,1,32,5.0,5,1,0,54,0


In [7]:
test.head(10)

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61
5,46232,Procurement,region_7,Bachelor's,m,sourcing,1,36,3.0,2,0,0,68
6,54542,Finance,region_2,Bachelor's,m,other,1,33,5.0,3,1,0,57
7,67269,Analytics,region_22,Bachelor's,m,sourcing,2,36,3.0,3,0,0,85
8,66174,Technology,region_7,Master's & above,m,other,1,51,4.0,11,0,0,75
9,76303,Technology,region_22,Bachelor's,m,sourcing,1,29,5.0,2,1,0,76


In [8]:
sub.head(5)

,employee_id,is_promoted
0,8724,0
1,74430,0
2,72255,0
3,38562,0
4,64486,0


In [9]:
train.shape
test.shape

(54808, 14)

(23490, 13)

In [10]:
for i in train.columns:
    print(i, train[i].unique())

employee_id [65438 65141  7513 ... 13918 13614 51526]
department ['Sales & Marketing' 'Operations' 'Technology' 'Analytics' 'R&D'
 'Procurement' 'Finance' 'HR' 'Legal']
region ['region_7' 'region_22' 'region_19' 'region_23' 'region_26' 'region_2'
 'region_20' 'region_34' 'region_1' 'region_4' 'region_29' 'region_31'
 'region_15' 'region_14' 'region_11' 'region_5' 'region_28' 'region_17'
 'region_13' 'region_16' 'region_25' 'region_10' 'region_27' 'region_30'
 'region_12' 'region_21' 'region_8' 'region_32' 'region_6' 'region_33'
 'region_24' 'region_3' 'region_9' 'region_18']
education ["Master's & above" "Bachelor's" nan 'Below Secondary']
gender ['f' 'm']
recruitment_channel ['sourcing' 'other' 'referred']
no_of_trainings [ 1  2  3  4  7  5  6  8 10  9]
age [35 30 34 39 45 31 33 28 32 49 37 38 41 27 29 26 24 57 40 42 23 59 44 50
 56 20 25 47 36 46 60 43 22 54 58 48 53 55 51 52 21]
previous_year_rating [ 5.  3.  1.  4. nan  2.]
length_of_service [ 8  4  7 10  2  5  6  1  3 16  9 11 26 

In [15]:
train["education"].replace(np.nan, "Bachelor's", inplace = True)
test["education"].replace(np.nan, "Bachelor's", inplace = True)

train["education"].replace("Master's & above", 3, inplace = True)
test["education"].replace("Master's & above", 3, inplace = True)
train["education"].replace("Bachelor's", 2, inplace = True)
test["education"].replace("Bachelor's", 2, inplace = True)
train["education"].replace("Below Secondary", 1, inplace = True)
test["education"].replace("Below Secondary", 1, inplace = True)

train["previous_year_rating"].replace(np.nan, 3., inplace = True)
test["previous_year_rating"].replace(np.nan, 3., inplace = True)

In [20]:
train["sum_metric"] = train["awards_won?"] + train["KPIs_met >80%"] + train["previous_year_rating"]
test["sum_metric"] = train["awards_won?"] + test["KPIs_met >80%"] + test["previous_year_rating"]

In [22]:
train["total_s"] = train["avg_training_score"] * train["no_of_trainings"]
test["total_s"] = test["avg_training_score"] * test["no_of_trainings"]